<a href="https://colab.research.google.com/github/CodeAlchemyML/text_to_video/blob/main/Longer_video_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers sentencepiece transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 25.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinu

In [2]:
!pip install torch

**Importing required libraries:**

In [3]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import numpy as np
from diffusers.utils import export_to_video
from google.colab import files

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

**Creating a pipeline:**

In [4]:
pipeline = DiffusionPipeline.from_pretrained( 'damo-vilab/text-to-video-ms-1.7b', torch_dtype = torch.float16, variant = 'fp16' )
pipeline.scheduler = DPMSolverMultistepScheduler.from_config( pipeline.scheduler.config )
pipeline.enable_model_cpu_offload()
pipeline.enable_vae_slicing()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

...

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

**Testing the pipeline:**

In [5]:
prompt = 'Spiderman is dancing.'
video_frames = pipeline( prompt, num_inference_steps = 25 ).frames
print( video_frames[0].shape )
spatial_frames_list = [ frame[ :, :, : ] for frame in video_frames[ 0 ] ]
repeat_count = int( 300 / len( spatial_frames_list ) )
repeated_frames = np.repeat( spatial_frames_list, repeat_count, axis = 0 )
spatial_frames = repeated_frames[ : 300 ]
spatial_frames = np.array( spatial_frames )
video_path = export_to_video( spatial_frames )
files.download( video_path )

  0%|          | 0/25 [00:00<?, ?it/s]

(16, 256, 256, 3)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Defining a function:**

In [6]:
def generate_video( prompt ):
  video_frames = pipeline( prompt, num_inference_steps = 25 ).frames
  spatial_frames_list = [ frame[ :, :, : ] for frame in video_frames[ 0 ] ]
  repeat_count = int( 300 / len( spatial_frames_list ) )
  repeated_frames = np.repeat( spatial_frames_list, repeat_count, axis = 0 )
  spatial_frames = repeated_frames[ : 300 ]
  spatial_frames = np.array( spatial_frames )
  video_path = export_to_video( spatial_frames )
  files.download( video_path )

**Making a basic bot with the help of basic iteration:**

In [7]:
print( "Hi!! I'm a Video generation bot. I generate videos based on your text prompt." )
prompt = input( 'Enter your prompt (For ending the chat, enter "end"):' )
while ( prompt != 'end' ):
  generate_video( prompt )
  prompt = input( 'Enter your prompt (For ending the chat, enter "end"):' )

else:
  print( 'Thank you!' )

Hi!! I'm a Video generation bot. I generate videos based on your text prompt.
Enter your prompt (For ending the chat, enter "end"):Superman is surfing


  0%|          | 0/25 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Enter your prompt (For ending the chat, enter "end"):end
Thank you!
